In [40]:
from collections import deque
import itertools
from itertools import product
import sys

In [89]:
class Symbol:
    def __init__(self, premise):
        self.premise = premise

    def __str__(self):
        return f"{self.premise}"
        
class Node:
    def __init__(self, center):
        self.left = None
        self.right = None
        self.center = center

    
class And:
    def __init__(self, *operands):
        self.operands = operands
        self.operator = "AND"

    def __rpr__(self):
        cls = self.__class__.__name__
        return f"{cls}({self.operands[0]},{self.operands[1]}, {self.operator})"
    
    def __str__(self):
        return f"{self.__class__.__name__}"
        
class Or:
    def __init__(self, *operands):
        self.operands = operands
        self.operator = "OR"

    def __rpr__(self):
        cls = self.__class__.__name__
        return f"{cls}({self.operands[0]},{self.operands[1]}, {self.operator})"
    
    def __str__(self):
        return f"{self.__class__.__name__}"


class If:
    def __init__(self, *operands):
        self.operands = operands
        self.operator = "If"

    def __rpr__(self):
        cls = self.__class__.__name__
        return f"{cls}({self.operands[0]},{self.operands[1]}, {self.operator})"

    def __str__(self):
        return f"{self.__class__.__name__}"


class Not:
    def __init__(self, operands):
        self.operands = operands
        self.operator = "Not"

    def __rpr__(self):
        cls = self.__class__.__name__
        return f"{cls}({self.operands}, {self.operator})"

    def __str__(self):
        return f"{self.__class__.__name__}"

        
class Knowledge:
    def __init__(self):
        self.storage = []
        self.symb_true = []
        self.symb_false = []
        self.operator_list = ["And","Or","Not","If","Iff"]

    def add(self, *args):
        if isinstance(args[0], Symbol):
            if len(args) > 1:
                if args[1] == True:
                    self.symb_true.append(args[0])
                else:
                    self.symb_false.append(args[0])
            else:
                self.symb_true.append(args[0])

        elif isinstance(args[0], Not):
            self.symb_false.append(args[0].operands)

        else:
            node = self.__build_tree(args[0])
            self.storage.append(node)

    def symbset_filter(self, s_list):

        fixed_keys = set(self.symb_true + self.symb_false)
        variable_keys = [key for key in s_list if key not in fixed_keys]
        result_concise = []
        if variable_keys:
  
            for combo in itertools.product([True, False], repeat=len(variable_keys)):
                dict_combo = {key: True for key in self.symb_true} 
                dict_combo.update({key: False for key in self.symb_false})  
                
                for key, value in zip(variable_keys, combo):
                    dict_combo[key] = value
                result_concise.append(dict_combo)
        else:
            dict_combo = {key: True for key in self.symb_true}
            dict_combo.update({key: False for key in self.symb_false})
            result_concise = [dict_combo]

        for re in result_concise:
            for key, values in re.items():
                print(f"{str(key)} : {values}")
        return result_concise

    def __build_tree(self, sentence):
        if isinstance(sentence, And):
            node = Node(sentence.__class__.__name__)
            node.left = self.__build_tree(sentence.operands[0])
            node.right = self.__build_tree(sentence.operands[1])
        elif isinstance(sentence, If):
            node = Node(sentence.__class__.__name__)
            node.left = self.__build_tree(sentence.operands[0])
            node.right = self.__build_tree(sentence.operands[1])
        elif isinstance(sentence, Or):
            node = Node(sentence.__class__.__name__)
            node.left = self.__build_tree(sentence.operands[0])
            node.right = self.__build_tree(sentence.operands[1])
        elif isinstance(sentence, Not):
            node = Node(sentence.__class__.__name__)
            node.left = self.__build_tree(sentence.operands)
        else:
            node = Node(sentence)
        return node

    def get_symbolFromStatement(self, statement):
        symbol_list = [x for x in statement if x not in self.operator_list]
        return symbol_list

    def inorder_traversal(self, node, statement):
        if node:
            self.inorder_traversal(node.left, statement)
            if isinstance(node, Symbol):
                statement.append(node)            
            else:
                statement.append(node.center)
            self.inorder_traversal(node.right, statement)

    def value_convert(self, statement, chunk):
        new_list = []
        for item in statement:
            if item in chunk:
                new_list.append(chunk[item])
            else:
                new_list.append(item)
        return new_list

    def not_rule(self, operand):
        if operand == True:
            return False
        else:
            return True

    def and_rule(self, *operand):
        if (operand[0] == True) and (operand[1] == True):
            return True
        else:
            return False
    
    def or_rule(self, *operand):
        if (operand[0] == False) and (operand[1] == False):
            return False
        else:
            return True

    def if_rule(self, *operand):
        if operand[1] == True:
            return True
        if (operand[0] == False) and (operand[1] == False):
            return True
        else:
            return False

    def iff_rule(self, *operand):
        if (operand[0] == True) and (operand[1] == True):
            return True
        if (operand[0] == False) and (operand[1] == False):
            return True
        else:
            return False
    
    def inference(self, value_symb):
        inferenced_symb = value_symb.copy()
        i = 0
        while i < len(inferenced_symb):
            if (inferenced_symb[i]== "Not"):
                operand = inferenced_symb[i-1]
                result = self.not_rule(operand)
                inferenced_symb[i-1] = result
                del inferenced_symb[i]
                print(inferenced_symb)
                i=0
            else:
                i+=1
        i = 0
        while i < len(inferenced_symb):
            if (inferenced_symb[i] == "And")or(inferenced_symb[i] == "Or"):
                operand_1 = inferenced_symb[i-1]
                operand_2 = inferenced_symb[i+1]
                if inferenced_symb[i] == "And":
                    result = self.and_rule(operand_1, operand_2)
                elif inferenced_symb[i] == "Or":
                    result = self.or_rule(operand_1, operand_2)
                inferenced_symb[i-1] = result
                del inferenced_symb[i]
                del inferenced_symb[i]
                print(inferenced_symb)
                i=0
            elif (inferenced_symb[i] == "If")or(inferenced_symb[i] == "Iff"):
                operand_1 = inferenced_symb[i-1]
                operand_2 = inferenced_symb[i+1]
                if inferenced_symb[i] == "If":
                    result = self.if_rule(operand_1, operand_2)
                elif inferenced_symb[i] == "Iff":
                    result = self.iff_rule(operand_1, operand_2)
                inferenced_symb[i-1] = result
                del inferenced_symb[i]
                del inferenced_symb[i]
                print(inferenced_symb)
                i=0
            else:
                i+=1
        return inferenced_symb
                

    def analyzer(self, *args):
        self.add(*args)
        for node in self.storage:
            statement = []
            self.inorder_traversal(node, statement)
            # print(statement)
            symbols = [x for x in statement if x not in self.operator_list]
            # for s in symbols:
            #     print(str(s))
            # print("----")
            symbols_combination = self.symbset_filter(symbols)
            for chunk in symbols_combination:
                value_symb = self.value_convert(statement, chunk)
                print(value_symb)
                print("-------")
                decision = self.inference(value_symb)
                if decision[0] == False:
                    print(f"KB is not entail {str(args[0])} => {args[1]}")
                    return
        print(f"KB is entail: {str(args[0])} => {args[1]}")
            

In [90]:
P = Symbol("Andy brush his teeth")
Q = Symbol("Andy wash his face")
R = Symbol("Andy turn up the car")
S = Symbol("Andy goes to campus")
sentence_1 = If(And(And(P,Not(Q)),R),S)
# sentence_2 = S
sentence_3 = R
sentence_4 = P
KB = Knowledge()
KB.add(sentence_1)
# KB.add(sentence_2)
KB.add(sentence_3)
KB.add(sentence_4)
KB.analyzer(S, False)

Andy turn up the car : True
Andy brush his teeth : True
Andy goes to campus : False
Andy wash his face : True
Andy turn up the car : True
Andy brush his teeth : True
Andy goes to campus : False
Andy wash his face : False
[True, 'And', True, 'Not', 'And', True, 'If', False]
-------
[True, 'And', False, 'And', True, 'If', False]
[False, 'And', True, 'If', False]
[False, 'If', False]
[True]
[True, 'And', False, 'Not', 'And', True, 'If', False]
-------
[True, 'And', True, 'And', True, 'If', False]
[True, 'And', True, 'If', False]
[True, 'If', False]
[False]
KB is not entail Andy goes to campus => False


In [86]:
P11 = Symbol("P11")
P21 = Symbol("P21")
P31 = Symbol("P31")
P41 = Symbol("P41")
P12 = Symbol("P12")
P13 = Symbol("P13")
P14 = Symbol("P14")
P22 = Symbol("P22")
P23 = Symbol("P23")
P24 = Symbol("P24")
P32 = Symbol("P32")
P33 = Symbol("P33")
P34 = Symbol("P34")
P42 = Symbol("P42")
P43 = Symbol("P43")
P44 = Symbol("P44")

B11 = Symbol("B11")
B21 = Symbol("B21")
B31 = Symbol("B31")
B41 = Symbol("B41")
B12 = Symbol("B12")
B13 = Symbol("B13")
B14 = Symbol("B14")
B22 = Symbol("B22")
B23 = Symbol("B23")
B24 = Symbol("B24")
B32 = Symbol("B32")
B33 = Symbol("B33")
B34 = Symbol("B34")
B42 = Symbol("B42")
B43 = Symbol("B43")
B44 = Symbol("B44")
S12 = Symbol("S11")
W13 = Symbol("W13")
W22 = Symbol("W22")
sym_list = [B11,B21,B31,B41,B12,B13,B14,B22,B23,B24,B32,B33,B34,B42,B43,B44,
            P11,P21,P31,P41,P12,P13,P14,P22,P23,P24,P32,P33,P34,P42,P43,P44]
# sym_list = [B21,P11,P21,P31,P22, S12, W13, W22]
KB = Knowledge(sym_list)

In [87]:
KB.add(Not(P11))
KB.add(If(B21,Or(P31,P22)))
KB.add(S12)
KB.add(If(S12,Or(W13,W22)))
KB.analyzer(P22,False)

S11 : True
P11 : False
P22 : False
B21 : True
P31 : True
S11 : True
P11 : False
P22 : False
B21 : True
P31 : False
S11 : True
P11 : False
P22 : False
B21 : False
P31 : True
S11 : True
P11 : False
P22 : False
B21 : False
P31 : False
[True, 'If', True, 'Or', False]
-------
[True, 'Or', False]
[True]
[True, 'If', False, 'Or', False]
-------
[False, 'Or', False]
[False]
KB is not entail P22 => False
